In [1]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import time

#Ist time_since_last_change relevant
# Was machen wir, wenn change überfällig ist

In [2]:
def calc_precision(preds,y,days_diff):
    return np.sum(np.abs(np.floor(preds)-y)<=days_diff)/len(y)

In [3]:
df_pop= pd.read_csv('../../../data/popular_data_with_features3.csv', index_col=0)
df_pop["timestamp"] = pd.to_datetime(df_pop["timestamp"]).dt.tz_localize(None)
df_pop.head()

/home/vpr/miniconda3/envs/mp/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,infobox_key,property_name,page_title,template,timestamp,day_of_year,day_of_month,day_of_week,month_of_year,quarter_of_year,...,is_month_end,is_quarter_start,is_quarter_end,days_since_last_change,days_since_last_2_changes,days_since_last_3_changes,days_until_next_change,days_between_last_and_2nd_to_last_change,mean_change_frequency_all_previous,mean_change_frequency_last_3
0,27792315-0,genre,Wildflowers_(Tom_Petty_album),infobox_album,2015-07-10,191,10,4,7,3,...,False,False,False,0,0,0,55,0,0.000000,0.000000
1,27792315-0,genre,Wildflowers_(Tom_Petty_album),infobox_album,2015-09-03,246,3,3,9,3,...,False,False,False,55,0,0,775,0,55.000000,0.000000
2,27792315-0,genre,Wildflowers_(Tom_Petty_album),infobox_album,2017-10-17,290,17,1,10,4,...,False,False,False,775,830,0,15,55,415.000000,0.000000
3,27792315-0,genre,Wildflowers_(Tom_Petty_album),infobox_album,2017-11-01,305,1,2,11,4,...,False,False,False,15,790,845,109,775,281.666667,281.666667
4,27792315-0,genre,Wildflowers_(Tom_Petty_album),infobox_album,2018-02-18,49,18,6,2,1,...,False,False,False,109,124,899,0,15,238.500000,299.666667


In [4]:
top_changes = df_pop.groupby(['infobox_key', 'property_name']).count()["day_of_year"].sort_values(ascending=False)

In [5]:
buckets_sizes = [0,10,50,100,500,1000,99999]
buckets = []
for i in range(len(buckets_sizes)-1):
    low=buckets_sizes[i]
    high=buckets_sizes[i+1]
    keys=top_changes.loc[(top_changes>=low) & (top_changes<high)].index.to_list()
    buckets.append({"low":low,"high":high,"size":len(keys),"keys":keys})
buckets

[{'low': 0,
  'high': 10,
  'size': 206229,
  'keys': [('413085013-0', 'misc'),
   ('293117094-0', 'currentclub'),
   ('238601551-0', 'caps(goals)'),
   ('412102974-0', 'club-update'),
   ('244058879-0', 'currentclub'),
   ('100189266-0', 'currentclub'),
   ('115466634-0', 'genre'),
   ('117848872-0', 'birth_date'),
   ('301365044-0', 'currentclub'),
   ('101221329-0', 'position'),
   ('398858359-0', 'birth_date'),
   ('107724925-0', 'image'),
   ('244196641-0', 'currentclub'),
   ('311153561-0', 'currentclub'),
   ('309016451-0', 'pcupdate'),
   ('311199505-0', 'birth_date'),
   ('301659387-0', 'currentclub'),
   ('30895665-0', 'released'),
   ('286266702-0', 'genre'),
   ('286339474-0', 'reviews'),
   ('24421657-0', 'image'),
   ('24407328-0', 'reviews'),
   ('268452912-0', 'name'),
   ('263636559-0', 'pcupdate'),
   ('421233858-0', 'occupation'),
   ('101221329-0', 'pcupdate'),
   ('115434673-0', 'birth_place'),
   ('105677502-0', 'genre'),
   ('27050398-0', 'genre'),
   ('30083953-

In [8]:
def generate_predictions(key_list):
    predictions = []
    accuracys = []
    feature_importances = []
    for key in tqdm(key_list):
        sample = df_pop[(df_pop['infobox_key'] == key[0]) &
                        (df_pop['property_name'] == key[1])]
        # sample = df_pop.iloc[start:start+iloc]
        # trainsize=round(len(sample)*0.8) # 20 % testdata
        sample.drop(columns=["infobox_key", "page_title",
                    "property_name", "template"], inplace=True)

        trainset = sample[sample["timestamp"] < datetime.datetime(2018, 9, 1)]
        testset = sample[sample["timestamp"] >= datetime.datetime(2018, 9, 1)]
        if len(trainset) == 0 or len(testset) == 0:
            continue
        X_train = trainset.drop(columns=['days_until_next_change', 'timestamp'])
        y_train = trainset['days_until_next_change']

        X_test = testset.drop(columns=['days_until_next_change', 'timestamp'])
        y_test = testset['days_until_next_change']
        # x_last = sample.iloc[-1].drop(['days_until_next_change','timestamp']).values.reshape(1,-1)

        # x_last_timestamp = datetime.datetime.strptime(sample.iloc[-1]["timestamp"],"%Y-%m-%d").date()
        clf = RandomForestClassifier(random_state=0, n_estimators=10)
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)
        predictions.append(preds)
        accuracys.append(calc_precision(preds, y_test, 1))
        feature_importances.append(clf.feature_importances_)

    return predictions, accuracys, feature_importances


In [9]:
import warnings
warnings.filterwarnings('ignore')
predictions=[]
accuracys=[]
feature_importances=[]
for key_list in tqdm(buckets[1:]):

    preds,accs,feature_imps=generate_predictions(key_list["keys"][:100])
    predictions.append(preds)
    accuracys.append(accs)
    feature_importances.append(feature_imps)


  0%|          | 0/5 [00:00<?, ?it/s]


ValueError: Found array with 0 sample(s) (shape=(0, 15)) while a minimum of 1 is required.

In [ ]:
import scipy
for i,acc in enumerate(accuracys):

    print("<",buckets[i+1]["high"],scipy.stats.describe(acc))

In [ ]:
columns=['day_of_year', 'day_of_month', 'day_of_week', 'month_of_year',
       'quarter_of_year', 'is_month_start', 'is_month_end', 'is_quarter_start',
       'is_quarter_end', 'days_since_last_change', 'days_since_last_2_changes',
       'days_since_last_3_changes', 'days_between_last_and_2nd_to_last_change',
       'mean_change_frequency_all_previous', 'mean_change_frequency_last_3']
feature_importances_df=pd.DataFrame(feature_importances,columns=columns)
feature_importances_df.describe()
feature_importances_df.to_csv("rf_feature_importances_1000rfs.csv")

In [ ]:
preds_df=pd.DataFrame(ilocs.index.to_list(),columns=["infobox_key","property_name"])
preds_df["pred"]=preds
preds_df